In [2]:
import networkx as nx
from pathlib import Path

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from flinspect.parse_tree import ParseTree
from flinspect.parse_forest import ParseForest

In [14]:
build_dirs = [
    "/glade/work/altuntas/turbo-stack/bin/flang_ptree/FMS2/",
    "/glade/work/altuntas/turbo-stack/bin/flang_ptree/MOM6-infra",
    "/glade/work/altuntas/turbo-stack/bin/flang_ptree/MOM6"
]

parse_tree_paths = [f for dir in build_dirs for f in Path(dir).glob("*_ptree")]

In [15]:
pf = ParseForest(parse_tree_paths)

In [16]:
nr = pf.registry

# Module Dependency Analysis

In [17]:
G = pf.get_module_dependency_graph()

Skipped 8 modules with unknown parse tree paths: ['netcdf', 'iso_c_binding', 'mpi', 'iso_fortran_env', 'netcdf_nf_data', 'netcdf_nf_interfaces', 'netcdf4_nf_interfaces', 'omp_lib']


In [18]:
# Determine source paths via the path_names files in build dirs
# This will be used to color nodes and edges based on where modules are located
path_names = {}
for build_dir in build_dirs:
    path_names_file = Path(build_dir)/"path_names"
    with open(path_names_file) as f:
        for line in f:
            src_file_path = Path(line.strip())
            path_names[src_file_path.stem.lower()] = src_file_path.as_posix()

In [19]:
# Find all nodes where /FMS/ is in the source path
fms_nodes = [node for node in G.nodes() if "/FMS/" in path_names.get(G.nodes[node].get('source_name','').lower(),'')]

In [20]:
# Find all fms_nodes that have no incoming edges (i.e., no dependencies)
fms_root_nodes = [node for node in fms_nodes if G.in_degree(node) == 0]

for fms_root_node in fms_root_nodes:
    print(fms_root_node, path_names.get(G.nodes[fms_root_node].get('source_name','').lower(),''))

mpp_pset_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/mpp/mpp_pset.F90
drifters_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/drifters/drifters.F90
grid_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/mosaic/grid.F90
fms_yaml_output_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/parser/fms_yaml_output.F90
fm_yaml_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/field_manager/fm_yaml.F90
fms /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/libFMS.F90
time_interp_external_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/time_interp/time_interp_external.F90
constantsr4_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/constants4/constantsr4.F90


In [21]:
# Find all FMS nodes that is not needed by any other FMS nodes
fms_leaf_nodes = [node for node in fms_nodes if G.out_degree(node) == 0]

for fms_leaf_node in fms_leaf_nodes:
    print(fms_leaf_node, path_names.get(G.nodes[fms_leaf_node].get('source_name','').lower(),''))

platform_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/platform/platform.F90
cloud_interpolator_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/drifters/cloud_interpolator.F90
monin_obukhov_inter /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/monin_obukhov/monin_obukhov_inter.F90
mersennetwister_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/random_numbers/mersennetwister.F90
fms_yaml_output_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/parser/fms_yaml_output.F90
fm_yaml_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/field_manager/fm_yaml.F90
yaml_parser_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/parser/yaml_parser.F90
tridiagonal_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/tridiagonal/tridiagonal.F90


In [22]:
# Find all MOM6 nodes:
mom6_nodes = [node for node in G.nodes() if "/MOM6/" in path_names.get(G.nodes[node].get('source_name','').lower(),'')]

# Find all FMS nodes that is not needed by any other MOM6 nodes recursively (i.e. transitive closure)
mom6_deps = set()
for mom6_node in mom6_nodes:
    mom6_deps.update(nx.descendants(G, mom6_node))

# Find all FMS nodes that are not needed by any other MOM6 nodes
fms_leaf_nodes = [node for node in fms_nodes if all(pred not in mom6_deps for pred in G.predecessors(node))]

for fms_leaf_node in fms_leaf_nodes:
    print(fms_leaf_node, path_names.get(G.nodes[fms_leaf_node].get('source_name','').lower(),''))

mpp_pset_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/mpp/mpp_pset.F90
drifters_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/drifters/drifters.F90
drifters_core_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/drifters/drifters_core.F90
cloud_interpolator_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/drifters/cloud_interpolator.F90
drifters_input_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/drifters/drifters_input.F90
drifters_comm_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/drifters/drifters_comm.F90
drifters_io_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/drifters/drifters_io.F90
monin_obukhov_mod /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/monin_obukhov/monin_obukhov.F90
monin_obukhov_inter /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/monin_obukhov/monin_obukhov_inter.F90
grid_mod /glade/work/altuntas/turbo-stack/dev

# Call Graph Analysis

In [23]:
path_names = {}
for build_dir in build_dirs:
    path_names_file = Path(build_dir)/"path_names"
    with open(path_names_file) as f:
        for line in f:
            src_file_path = Path(line.strip())
            path_names[src_file_path.stem.lower()] = src_file_path.as_posix()

In [24]:
gsc = pf.get_call_graph()

Total unfound calls across all parse trees: 195
Total unfound function calls across all parse trees: 78829


In [25]:
# Find all FMS subroutines and MOM6 subroutines
all_fms_routines = []
all_mom6_routines = []
for node in gsc.nodes():
    try:
        module_path = node.program_unit.parse_tree_path
        source_path = path_names.get(module_path.stem.lower(),'')
    except AttributeError:
        continue
    if "/MOM6/" in source_path:
        all_mom6_routines.append(node)
    elif "/FMS/" in source_path:
        all_fms_routines.append(node)

In [26]:
len(all_mom6_routines), len(all_fms_routines)

(3525, 2956)

In [27]:
def get_fms_subroutines_needed_by_mom6(gsc, all_mom6_routines):
    fms_subroutines_needed_by_mom6 = set()
    for mom6_routine in all_mom6_routines:
        descendants = nx.descendants(gsc, mom6_routine)
        for desc in descendants:
            try:
                module_path = desc.program_unit.parse_tree_path
                source_path = path_names.get(module_path.stem.lower(),'')
            except AttributeError:
                continue
            if "/FMS/" in source_path:
                fms_subroutines_needed_by_mom6.add(desc)
    return fms_subroutines_needed_by_mom6

In [28]:
fms_subroutines_needed_by_mom6 = get_fms_subroutines_needed_by_mom6(gsc, all_mom6_routines)

In [29]:
len(fms_subroutines_needed_by_mom6)

1485

In [30]:
fms_subroutines_NOT_needed_by_mom6 = [r for r in all_fms_routines if r not in fms_subroutines_needed_by_mom6]
len(fms_subroutines_NOT_needed_by_mom6)

1471

In [31]:
# find all FMS subroutines that are not needed by MOM6 and are not called by any other FMS subroutines
fms_subroutines_NOT_needed_by_mom6_and_not_called_by_fms = []
for routine in fms_subroutines_NOT_needed_by_mom6:
    callers = list(gsc.predecessors(routine))
    called_by_fms = False
    for caller in callers:
        try:
            module_path = caller.program_unit.parse_tree_path
            source_path = path_names.get(module_path.stem.lower(),'')
        except AttributeError:
            continue
        if "/FMS/" in source_path:
            called_by_fms = True
            break
    if not called_by_fms:
        fms_subroutines_NOT_needed_by_mom6_and_not_called_by_fms.append(routine)

In [32]:
len(fms_subroutines_NOT_needed_by_mom6_and_not_called_by_fms)

608

In [33]:
for routine in fms_subroutines_NOT_needed_by_mom6_and_not_called_by_fms:
    if 'mpp_io' in routine.program_unit.name:
        print(routine.name)

mpp_get_time_axis
mpp_flush
mpp_write_axis_data
mpp_copy_meta_global
mpp_copy_meta_axis
mpp_copy_meta_field
fillin_fieldtype
mpp_get_default_calendar
mpp_get_field_index
mpp_get_axis_index
mpp_get_axis_by_name
mpp_get_axis_length
mpp_get_recdimid
mpp_get_ncid
mpp_get_axis_id
mpp_get_field_id
mpp_get_att_type
mpp_get_att_real
mpp_get_field_name
mpp_io_clock_on
mpp_get_maxunits
do_cf_compliance


In [34]:
fms_subroutines_NOT_needed_by_mom6_and_not_called_by_fms[10:]

[Subroutine('mpp_pset_stack_reset'),
 Subroutine('mpp_pset_print_stack_chksum'),
 Subroutine('mpp_pset_get_root_pelist'),
 Subroutine('diag_axis_add_attribute_scalar_r'),
 Subroutine('diag_axis_add_attribute_scalar_i'),
 Subroutine('diag_axis_add_attribute_scalar_c'),
 Subroutine('fms_sort_this'),
 Subroutine('c_free'),
 Subroutine('fms_f2c_string'),
 Subroutine('drifters_new'),
 Subroutine('drifters_copy_new'),
 Subroutine('drifters_set_domain'),
 Subroutine('drifters_set_pe_neighbors'),
 Subroutine('drifters_push_2'),
 Subroutine('drifters_push_3'),
 Subroutine('drifters_set_field_2d'),
 Subroutine('drifters_set_field_3d'),
 Subroutine('drifters_save'),
 Subroutine('drifters_distribute'),
 Subroutine('drifters_write_restart'),
 Subroutine('drifters_set_v_axes'),
 Subroutine('drifters_set_domain_bounds'),
 Subroutine('drifters_print_checksums'),
 Subroutine('drifters_comm_set_data_bounds'),
 Subroutine('drifters_comm_set_comp_bounds'),
 Subroutine('tranlon'),
 Subroutine('interp_1d_1d

In [35]:
#group unnecessary subroutines by modules they belong to:
from collections import defaultdict
unnecessary_subroutines_by_module = defaultdict(list)
for subroutine in fms_subroutines_NOT_needed_by_mom6:
    module_path = subroutine.program_unit.parse_tree_path
    source_path = path_names.get(module_path.stem.lower(),'')
    unnecessary_subroutines_by_module[source_path].append(subroutine)

In [36]:
# sort modules by ratio of unnecessary subroutines to total subroutines
module_unnecessary_ratio = []
for module_path, subroutines in unnecessary_subroutines_by_module.items():
    total_subroutines = [r for r in all_fms_routines if path_names.get(r.program_unit.parse_tree_path.stem.lower(),'') == module_path]
    ratio = len(subroutines) / len(total_subroutines)
    module_unnecessary_ratio.append((module_path, ratio, len(subroutines), len(total_subroutines)))

module_unnecessary_ratio.sort(key=lambda x: x[1], reverse=True)
for module in module_unnecessary_ratio[:30]:
    print(f"Module: {module[0]}, Unnecessary Ratio: {module[1]:.2f}, Unnecessary Count: {module[2]}, Total Count: {module[3]}")

Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/mpp/mpp_pset.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 28, Total Count: 28
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/drifters/drifters.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 21, Total Count: 21
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/drifters/drifters_comm.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 8, Total Count: 8
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/monin_obukhov/monin_obukhov.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 26, Total Count: 26
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/mosaic/grid.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 18, Total Count: 18
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/diag_manager/fms_diag_bbox.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 10, Total Count: 10
Module: /glade/work/altuntas/turbo-stack/dev-utils/..

In [37]:
# sort modules by ratio of unnecessary subroutines to total subroutines
module_unnecessary_ratio = []
for module_path, subroutines in unnecessary_subroutines_by_module.items():
    total_subroutines = [r for r in all_fms_routines if path_names.get(r.program_unit.parse_tree_path.stem.lower(),'') == module_path]
    ratio = len(subroutines) / len(total_subroutines)
    module_unnecessary_ratio.append((module_path, ratio, len(subroutines), len(total_subroutines)))

module_unnecessary_ratio.sort(key=lambda x: x[1], reverse=True)
for module in module_unnecessary_ratio[:30]:
    print(f"Module: {module[0]}, Unnecessary Ratio: {module[1]:.2f}, Unnecessary Count: {module[2]}, Total Count: {module[3]}")

Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/mpp/mpp_pset.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 28, Total Count: 28
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/drifters/drifters.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 21, Total Count: 21
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/drifters/drifters_comm.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 8, Total Count: 8
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/monin_obukhov/monin_obukhov.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 26, Total Count: 26
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/mosaic/grid.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 18, Total Count: 18
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/diag_manager/fms_diag_bbox.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 10, Total Count: 10
Module: /glade/work/altuntas/turbo-stack/dev-utils/..

In [38]:
# sort modules by ratio of unnecessary subroutines to total subroutines
module_unnecessary_ratio = []
for module_path, subroutines in unnecessary_subroutines_by_module.items():
    total_subroutines = [r for r in all_fms_routines if path_names.get(r.program_unit.parse_tree_path.stem.lower(),'') == module_path]
    ratio = len(subroutines) / len(total_subroutines)
    module_unnecessary_ratio.append((module_path, ratio, len(subroutines), len(total_subroutines)))

module_unnecessary_ratio.sort(key=lambda x: x[2], reverse=True)
for module in module_unnecessary_ratio[:30]:
    print(f"Module: {module[0]}, Unnecessary Ratio: {module[1]:.2f}, Unnecessary Count: {module[2]}, Total Count: {module[3]}")

Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/mpp/mpp_domains.F90, Unnecessary Ratio: 0.49, Unnecessary Count: 338, Total Count: 686
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/fms/fms_io.F90, Unnecessary Ratio: 0.80, Unnecessary Count: 148, Total Count: 186
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/mpp/mpp_io.F90, Unnecessary Ratio: 0.63, Unnecessary Count: 101, Total Count: 161
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/exchange/xgrid.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 60, Total Count: 60
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/sat_vapor_pres/sat_vapor_pres_k.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 59, Total Count: 59
Module: /glade/work/altuntas/turbo-stack/dev-utils/../submodules/FMS/sat_vapor_pres/sat_vapor_pres.F90, Unnecessary Ratio: 1.00, Unnecessary Count: 56, Total Count: 56
Module: /glade/work/altuntas/turbo-stack/dev-utils/